In [8]:
# !pip install transformers torch
# !pip install 'accelerate>=0.26.0'

# นำเข้าโมดูลที่ต้องการ
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def askTyphoon2(prompt: str):
    # โหลดโมเดลและ tokenizer
    model_id = "scb10x/llama3.2-typhoon2-1b-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)
    # model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

    # เตรียมข้อความนำเข้า
    messages = [
        {"role": "system", "content": "คุณคือผู้ช่วย AI ที่ให้คำตอบอย่างดี"},
        {"role": "user", "content": prompt}
    ]
    # แปลงข้อความเป็น token (คืนค่าเป็น tensor โดยตรง)
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    
    # สร้าง attention_mask จาก input_ids (1 สำหรับ token จริง, 0 สำหรับ padding)
    attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)
    
    # กำหนดพารามิเตอร์สำหรับการสร้างข้อความ
    terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    
    # สร้างข้อความตอบกลับ
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,  # ส่ง attention_mask เข้าไป
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id  # กำหนด pad_token_id อย่างชัดเจน
    )
    
    # แปลงผลลัพธ์กลับเป็นข้อความ
    response = outputs[0][input_ids.shape[-1]:]
    answer = tokenizer.decode(response, skip_special_tokens=True)
    print(answer)

In [9]:
askTyphoon2("แต่งเรื่องกระต่ายกับเต่า")

ในหมู่บ้านเล็กๆ แห่งหนึ่ง มีกระต่ายชื่อ "เจ้ากระต่าย" และเต่าชื่อ "เจ้าเต่า" ทั้งสองเป็นเพื่อนรักและต้องการสร้างสรรค์สิ่งใหม่ๆ เพื่อให้บ้านของพวกเขาดูโดดเด่นยิ่งขึ้น

วันหนึ่ง กระต่ายได้ไอเดียเก๋ๆ ว่า "เราสามารถทำสวนแห่งความสุขในบ้านของเรา!" และด้วยความคิดสร้างสรรค์ของมัน กระต่ายก็ช่วยเต่าในการปลูกต้นไม้และดูแลสวนให้เต็มไปด้วยสีสัน

ส่วนเจ้าเต่าเอง ก็ไม่พลาดที่จะช่วยกระต่ายในการทำอาหารด้วยการช่วยเต่าปรุงอาหารอร่อยๆ ให้กระต่ายได้ทดลองชิม

เมื่อเวลาผ่านไป สวนแห่งความสุขของพวกเขาก็เต็มไปด้วยดอกไม้และผลไม้หลากหลายชนิด ทำให้ทั้งกระต่ายและเต่ารู้สึกมีความสุขอย่างมาก

ในที่สุด ทั้งสองเพื่อนรักได้เรียนรู้ว่าความสุขไม่ได้อยู่ที่สิ่งที่เราเห็น แต่คือการช่วยเหลือและแบ่งปันกัน ทำให้บ้านของพวกเขากลายเป็นสถานที่ที่เต็มไปด้วยความรักและการแบ่งปัน

เรื่องราวนี้ไม่เพียงแต่สร้างความสุขให้กับกระต่ายและเต่า แต่ยังทำให้พวกเขาได้เรียนรู้ถึงคุณค่าของการช่วยเหลือกันในชีวิตประจำวันอีกด้วย
